In [1]:
# Read namespaces
import os
import kubernetes
import ipywidgets as widgets
from IPython.display import display

ns_list = []
kubernetes.config.load_kube_config()
k8s_client = kubernetes.client.CoreV1Api()
namespaces = k8s_client.list_namespace(_request_timeout=(1, 5))
for namespace in namespaces.items:
    ns_name = namespace.metadata.name
    if ns_name[0:3] == "ci-":
        ns_list.append(ns_name)
print(f"Read {len(ns_list)} namespaces")
w = widgets.Dropdown(
    options=ns_list,
    description='Namespace:',
    disabled=False,
)
display(w)

Read 4 namespaces


Dropdown(description='Namespace:', options=('ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks', 'ci-di…

In [4]:
# Set namespaces
SUT_NAMESPACE = w.value
print(f"Using SUT namespace {SUT_NAMESPACE}")
BRANCH_NAME = SUT_NAMESPACE.replace("ci-ska-mid-itf-", "")
print(f"Using SUT branch {BRANCH_NAME}")
SKA001_NAMESPACE = f"ci-dish-lmc-ska001-{BRANCH_NAME}"
print(f"Using SKA 001 namespace {SKA001_NAMESPACE}")
SKA036_NAMESPACE = f"ci-dish-lmc-ska036-{BRANCH_NAME}"
print(f"Using SKA 036 namespace {SKA036_NAMESPACE}")

Using SUT namespace ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks
Using SUT branch at-2146-consolidate-jupyter-notebooks
Using SKA 001 namespace ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks
Using SKA 036 namespace ci-dish-lmc-ska036-at-2146-consolidate-jupyter-notebooks


In [5]:
# Set Up and Configure Device Proxies
print("Setting up parameters for SUT namespace {SUT_NAMESPACE}\n")

TANGO_HOST = f"tango-databaseds.{SUT_NAMESPACE}.svc.miditf.internal.skao.int:10000"
os.environ["TANGO_HOST"] = TANGO_HOST

RECEPTORS = ["SKA001", "SKA036"]

# Config files set up
DATA_DIR = "../../data"
TMC_CONFIGS = f"{DATA_DIR}/mid_telescope/tmc"
SCAN_FILE = f"{TMC_CONFIGS}/scan.json"
RELEASE_RESOURCES_FILE = f"{DATA_DIR}/release_resources.json"

ASSIGN_RESOURCES_FILE = f"{TMC_CONFIGS}/assign_resources.json"
CONFIGURE_SCAN_FILE = f"{TMC_CONFIGS}/configure_scan.json"

CBF_CONFIGS = f"{DATA_DIR}/mid_telescope/cbf"
DISH_CONFIG_FILE = f"{CBF_CONFIGS}/sys_params/load_dish_config.json"

print("Links for Taranta and QA Display\n")
print("SUT Links")
print(f"https://k8s.miditf.internal.skao.int/{SUT_NAMESPACE}/signal/display/")
print(f"https://k8s.miditf.internal.skao.int/{SUT_NAMESPACE}/taranta/devices")
print("\n")
print("Dish LMC links")
print(f"https://k8s.miditf.internal.skao.int/{SKA001_NAMESPACE}/taranta/devices")
print(f"https://k8s.miditf.internal.skao.int/{SKA036_NAMESPACE}/taranta/devices")

Setting up parameters for SUT namespace {SUT_NAMESPACE}

Links for Taranta and QA Display

SUT Links
https://k8s.miditf.internal.skao.int/ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks/signal/display/
https://k8s.miditf.internal.skao.int/ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks/taranta/devices


Dish LMC links
https://k8s.miditf.internal.skao.int/ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks/taranta/devices
https://k8s.miditf.internal.skao.int/ci-dish-lmc-ska036-at-2146-consolidate-jupyter-notebooks/taranta/devices


In [3]:
import json
import time

from tango import DeviceProxy

print("Setting up the device proxies")

# TMC proxies
tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
tmc_csp_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")
tmc_csp_subarray = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
tmc_subarray = DeviceProxy("ska_mid/tm_subarray_node/1")

# CSP.LMC proxies
csp_control = DeviceProxy("mid-csp/control/0")
csp_subarray = DeviceProxy("mid-csp/subarray/01")

# CBF proxies
cbf_controller = DeviceProxy("mid_csp_cbf/sub_elt/controller")
cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")

# Dish Leaf Proxies
dish_leaf_node_ska001 = DeviceProxy("ska_mid/tm_leaf_node/d0001")
dish_leaf_node_ska036 = DeviceProxy("ska_mid/tm_leaf_node/d0036")

# SDP Proxies
sdp_subarray = DeviceProxy("mid-sdp/subarray/01")

# Set devices to adminMode = ONLINE
csp_control.adminMode = 0
csp_subarray.adminMode = 0

# Leaf Nodes
csp_subarray_leaf_node = "ska_mid/tm_leaf_node/csp_subarray01"
sdp_subarray_leaf_node = "ska_mid/tm_leaf_node/sdp_subarray01"
csp_master_leaf_node = "ska_mid/tm_leaf_node/csp_master"
csp_subarray_leaf_node_dp = DeviceProxy(csp_subarray_leaf_node)
sdp_subarray_leaf_node_dp = DeviceProxy(sdp_subarray_leaf_node)
csp_master_leaf_node_dp = DeviceProxy(csp_master_leaf_node)

time.sleep(2)
print("\nChecking admin mode after setting to ONLINE (0):")
print(f"  CSP Control: {csp_control.adminMode}")
print(f"  CSP Subarray: {csp_subarray.adminMode}")
print(f"  CBF Controller: {cbf_controller.adminMode}")
print(f"  CBF Subarray: {cbf_subarray.adminMode}")

# Set CBF Simulation mode to false and CBF timeout to 99s
csp_control.cbfSimulationMode = 0
csp_control.commandTimeout = 99

time.sleep(2)
print("\nChecking CBF Simulation Mode and CBF Timeout:")
print(f"  CBF Simulation Mode: {bool(csp_control.cbfSimulationMode)}")
print(f"  CBF Timeout: {csp_control.commandTimeout} sec")

Setting up the device proxies


ConnectionFailed: DevFailed[
DevError[
    desc = TRANSIENT CORBA system exception: TRANSIENT_ConnectFailed
  origin = void Tango::Connection::connect(const string&) at (/src/cppTango/src/client/devapi_base.cpp:609)
  reason = API_CorbaException
severity = ERR]

DevError[
    desc = Failed to connect to database on host 127.0.0.1 with port 10000
  origin = void Tango::Connection::connect(const string&) at (/src/cppTango/src/client/devapi_base.cpp:609)
  reason = API_CantConnectToDatabase
severity = ERR]
]

In [25]:
# Load the Dish Vcc Config / Init Sys Params
# NOTE: RUN TWICE
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

dish_config_json["tm_data_sources"][0] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"

print(f"dish_config_json file contents: \n{dish_config_json}")
tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))

time.sleep(2)
print(f"TMC CSP Master's Dish Vcc Config attribute value: \n{tmc_csp_master.dishVccConfig}")
print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{tmc_csp_master.sourceDishVccConfig}"
)

dish_config_json file contents: 
{'interface': 'https://schema.skao.int/ska-mid-cbf-initsysparam/1.0', 'tm_data_sources': ['car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata'], 'tm_data_filepath': 'instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json'}
TMC CSP Master's Dish Vcc Config attribute value: 
{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "dish_parameters": {"SKA001": {"vcc": 1, "k": 1}, "SKA036": {"vcc": 2, "k": 1}, "SKA063": {"vcc": 3, "k": 1}, "SKA100": {"vcc": 4, "k": 1}}}

TMC CSP Master's Source Dish Vcc Config attribute value: 
{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "tm_data_sources": ["car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"], "tm_data_filepath": "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"}


In [28]:
# Turn the Telescope On
import tango

print("Running the TelescopeOn command")
try:
    tmc_central_node.TelescopeOn()
except tango.DevFailed as t_err:
    err_msg: str = t_err.args[0].desc.strip()
    print(f"Could not run command: {err_msg}")
print("Ready")

Running the TelescopeOn command
Could not run command: ska_tmc_common.exceptions.CommandNotAllowed: Dish Vcc Config not Set. Please set using LoadDishCfg command. Current Telescope State is :ON
Ready


In [ ]:
# Check if telescope is on
p = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
#display(p)
n = 0
while int(tmc_central_node.telescopeState) != 0:
    print("Waiting for Telescope to come ON")
    n += 1
    p.value = n
    if n >= 100:
        break
    try:
        time.sleep(3)
    except KeyboardInterrupt:
        print("Abort")
    
if int(tmc_central_node.telescopeState) != 0:
    print("Telecope is ON")
else:
    print("Telecope is not ON")

In [ ]:
# Assign Resources
print(sdp_subarray.state())

time.sleep(3)

print(
    "Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned"
)

with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    assign_resources_json = json.load(f)
    assign_resources_json["dish"]["receptor_ids"] = RECEPTORS
    assign_resources_json["sdp"]["resources"]["receptors"] = RECEPTORS


print(f"\nassign_resources_json file contents: \n{assign_resources_json}")

tmc_subarray.AssignResources(json.dumps(assign_resources_json))

time.sleep(2)
print(f"\nCBF Subarray Observation State: {tmc_subarray.obsState}")
print(f"CBF Subarray Receptors : {cbf_subarray.receptors}")

In [ ]:
from astropy.time import Time
from tango import DeviceProxy

dish_deployments = [f"tango://tango-databaseds.{SKA001_NAMESPACE}.svc.miditf.internal.skao.int:10000/mid-dish/dish-manager/SKA001", 
                    f"tango://tango-databaseds.{SKA036_NAMESPACE}.svc.miditf.internal.skao.int:10000/mid-dish/dish-manager/SKA036"]


def get_tai_from_unix_s(unix_s: float) -> float:
    """
    Calculate atomic time in seconds from unix time in seconds.

    :param unix_s: Unix time in seconds

    :return: atomic time (tai) in seconds
    """
    astropy_time_utc = Time(unix_s, format="unix")
    return astropy_time_utc.unix_tai


for dish in dish_deployments:
    dish_manager_proxy = DeviceProxy(dish)
    current_pointing = dish_manager_proxy.achievedPointing
    current_az = current_pointing[1]
    current_el = current_pointing[2]

    current_time_tai_s = get_tai_from_unix_s(time.time() + 120)

    # Directions to move values
    az_dir = 1 if current_az < 350 else -1
    el_dir = 1 if current_el < 80 else -1

    track_table = [
        current_time_tai_s + 3,
        current_az + 1 * az_dir,
        current_el + 1 * el_dir,
        current_time_tai_s + 5,
        current_az + 2 * az_dir,
        current_el + 2 * el_dir,
        current_time_tai_s + 7,
        current_az + 3 * az_dir,
        current_el + 3 * el_dir,
        current_time_tai_s + 9,
        current_az + 4 * az_dir,
        current_el + 4 * el_dir,
        current_time_tai_s + 11,
        current_az + 5 * az_dir,
        current_el + 5 * el_dir,
    ]

    dish_manager_proxy.programTrackTable = track_table

In [ ]:
# Configure Scan¶
print("Running the Configure command: subarray obsstate should go to Ready")

with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    configure_scan_json = json.load(f)

print(f"\nconfigure_scan_json file contents: \n{configure_scan_json}")

print(json.dumps(configure_scan_json))

tmc_subarray.Configure(json.dumps(configure_scan_json))
    
sleep(12)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

In [ ]:
# TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    sdp_scan = d["sdp"]
    print(sdp_scan)
    sdp_subarray_leaf_node_dp.scan(json.dumps(sdp_scan))

sleep(10)
print(sdp_subarray_leaf_node_dp.sdpSubarrayObsState)

with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_scan = d["csp"]
    print(csp_scan)
    csp_subarray_leaf_node_dp.scan(json.dumps(csp_scan))

sleep(2)
print(csp_subarray_leaf_node_dp.cspSubarrayObsState)

In [ ]:
# TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
print("Running the End Scan command: subarray obsstate should go to Ready state")

sdp_subarray_leaf_node_dp.EndScan()
sleep(2)
csp_subarray_leaf_node_dp.EndScan()
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

In [ ]:
# TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
print("Running the End command: subarray obsstate should go to Idle state")

sdp_subarray_leaf_node_dp.End()
sleep(2)
csp_subarray_leaf_node_dp.End()

sleep(2)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

In [ ]:
# TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
)

sdp_subarray_leaf_node_dp.ReleaseAllResources()
sleep(2)
csp_subarray_leaf_node_dp.ReleaseAllResources()

sleep(2)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

In [ ]:
# Telescope Off
print("Running the TelescopeOff command")

tmc_central_node.TelescopeOff()

sleep(5)
print(f"TMC Central Node State: {tmc_central_node.State()}")
print(f"CSP Control State: {csp_control.State()}")
print(f"CBF Controller State: {cbf_controller.State()}")